In [1]:
# ============================================
# 1. INSTALACIÓN DE DEPENDENCIAS Y CONFIGURACIÓN
# ============================================

!pip install pystac-client planetary-computer geopandas shapely rioxarray xarray pyproj

from google.colab import drive
import os

# Montar Google Drive
print("Montando Google Drive...")
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 919.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 12.8 MB/s eta 0:00:00
Montando Google Drive...
Mounted at /content/drive


In [2]:

# ============================================
# 2. DEFINICIÓN MANUAL DE RUTAS
# ============================================

# Cambia estas rutas manualmente según tu estructura en Google Drive
GEOJSON_DIR = "/content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Json"
SENTINEL_DIR = "/content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel"

# Ruta fija del archivo GeoJSON
GEOJSON_PATH_MANUAL = os.path.join(GEOJSON_DIR, "Cauca_1523,4567_Tecnicafe.json")

In [3]:

# ============================================
# 3. IMPORTAR LIBRERÍAS Y DEFINIR PARÁMETROS
# ============================================

import geopandas as gpd
import rioxarray
import planetary_computer
from pystac_client import Client
import numpy as np
import xarray as xr
from shapely.geometry import shape, mapping
import json
from datetime import datetime
import requests

max_cloud_cover = 10
date_range = "2023-01-01/2024-12-31"

# Conectarse a Planetary Computer
pc = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [4]:
# ============================================
# 4. CARGAR GEOJSON Y PREPARAR GEOMETRÍA
# ============================================

with open(GEOJSON_PATH_MANUAL, 'r') as f:
    geojson_data = json.load(f)

geometries = [shape(feature['geometry']) for feature in geojson_data['features']]
gdf = gpd.GeoDataFrame(geometry=geometries, crs="EPSG:4326")
clip_geometry = gdf.geometry.iloc[0]

print("✅ GeoJSON cargado correctamente")

✅ GeoJSON cargado correctamente


In [6]:
# ============================================
# 5. BUSCAR Y DESCARGAR TODAS LAS BANDAS
# ============================================

try:
    bounds = gdf.total_bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    expanded_bounds = [
        bounds[0] - width * 0.2,
        bounds[1] - height * 0.2,
        bounds[2] + width * 0.2,
        bounds[3] + height * 0.2
    ]

    search = pc.search(
        collections=["sentinel-2-l2a"],
        bbox=expanded_bounds,
        datetime=date_range,
        query={"eo:cloud_cover": {"lt": max_cloud_cover}},
    )

    items = list(search.get_items())
    print(f"\nSe encontraron {len(items)} imágenes disponibles.")

    if len(items) > 0:
        selected = items[0].to_dict()
        signed_item = planetary_computer.sign(selected)

        # Obtener nombre base del archivo geojson y fecha del producto
        geojson_name = os.path.splitext(os.path.basename(GEOJSON_PATH_MANUAL))[0].replace(",", "_")
        fecha_imagen = selected['properties']['datetime'][:10].replace("-", "")

        # Crear carpeta específica para esta descarga
        carpeta_destino = os.path.join(SENTINEL_DIR, f"{geojson_name}_{fecha_imagen}")
        os.makedirs(carpeta_destino, exist_ok=True)

        for asset_name, asset_data in signed_item['assets'].items():
            href = asset_data['href']
            if href.endswith('.tif') or 'B' in asset_name:
                nombre_archivo = f"{geojson_name}_{fecha_imagen}_{asset_name}.tif"
                destino = os.path.join(carpeta_destino, nombre_archivo)

                try:
                    print(f"\n⬇️ Descargando {asset_name}...")
                    response = requests.get(href, stream=True)
                    with open(destino, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    print(f"✅ {asset_name} guardado en: {destino}")

                except Exception as e:
                    print(f"❌ Error con {asset_name}: {e}")
    else:
        print("No se encontraron imágenes para los criterios definidos.")

except Exception as e:
    raise Exception(f"Error al buscar o descargar imágenes Sentinel: {e}")


/usr/local/lib/python3.11/dist-packages/pystac_client/item_search.py:881: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(



Se encontraron 6 imágenes disponibles.

⬇️ Descargando B01...
✅ B01 guardado en: /content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel/Cauca_1523_4567_Tecnicafe_20241017/Cauca_1523_4567_Tecnicafe_20241017_B01.tif

⬇️ Descargando B02...
✅ B02 guardado en: /content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel/Cauca_1523_4567_Tecnicafe_20241017/Cauca_1523_4567_Tecnicafe_20241017_B02.tif

⬇️ Descargando B03...
✅ B03 guardado en: /content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel/Cauca_1523_4567_Tecnicafe_20241017/Cauca_1523_4567_Tecnicafe_20241017_B03.tif

⬇️ Descargando B04...
✅ B04 guardado en: /content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel/Cauca_1523_4567_Tecnicafe_20241017/Cauca_1523_4567_Tecnicafe_20241017_B04.tif

⬇️ Descargando B05...
✅ B05 guardado en: /content/drive/Shareddrives/DEV/Nuwa/Data/Modelo_NDVI/Sentinel/Cauca_1523_4567_Tecnicafe_20241017/Cauca_1523_4567_Tecnicafe_20241017_B05.tif

⬇️ Descargando B06...
✅ B06 guardado en: /co